# データの保存
後からデータを利用するために，**シミュレーション条件**と**シミュレーション結果**の2つを保存する必要がある．  
まずはそれらをcsvとyamlを使って保存するやり方を説明する．  

In [157]:
import scipy.integrate as integrate
import numpy as np

まずはデータを準備．  
前扱ったローレンツ方程式のシミュレーションです．  

In [158]:
# ローレンツ方程式のシミュレーション
def loorenz_list(t, X, p, r, b):
    """リスト"""
    x, y, z = X
    
    dx = -p*x + p*y
    dy = -x*z + r*x -y
    dz = x*y - b*z
    
    return [dx, dy, dz]

TIME_SPAN = 50
TIME_INTERVAL = 0.01

X0_list = [0, 4, 28]

p = 10
r = 28
b = 8/3

sol = integrate.solve_ivp(
    fun = loorenz_list,
    t_span = (0, TIME_SPAN),
    y0 = X0_list,
    t_eval=np.arange(0, TIME_SPAN, TIME_INTERVAL),
    args=(p, r, b)
)

****
## 条件を保存
読みやすくて再利用しやすいのでyaml形式で保存するのが良いと思う．  
yamlファイルについて： 

In [159]:
import yaml  # 標準では入ってない．pip imstall pyyamlかconda install pyyamlでインストール

まずは保存した条件を辞書にする．  

In [160]:
param = {
    'TIME_SPAN' : TIME_SPAN,
    'TIME_INTERVAL' : TIME_INTERVAL,
    'X0_init' : X0_list,
    'p' : p,
    'r' : r,
    'b' : b,
}
param

{'TIME_SPAN': 50,
 'TIME_INTERVAL': 0.01,
 'X0_init': [0, 4, 28],
 'p': 10,
 'r': 28,
 'b': 2.6666666666666665}

### yamlで保存
次の関数を使う．  
```python
yaml.dump(pythonオブジェクト, ファイル)
```

In [161]:
with open('parameter.yaml', "w") as f:  # 'w'は書き込むという意味
    yaml.dump(param, f)

### yamlの読み込み
`yaml.safe_load("yamlファイル")`で読み込む．  

In [162]:
with open('parameter.yaml') as f:
    param_read = yaml.safe_load(f)

もとのデータと一致しているか確認

In [163]:
param_read == param

True

### 注意
yamlにndarrayは保存できない．  
yaml以外の形式で保存するか，python標準のリストに変換してから保存する．  

****
## 結果の保存
いろいろなソフトで読み書きしやすいのでcsvで保存するのが一般的．  
ここでは`sol.t`と`sol.y`を保存する．  
### csvで保存
solve_ivpでは解はndarrayで返されるので，ndarrayをcsvに保存できる`np.savetxt()`を使う．  


In [164]:
# まずはヘッダーを準備
header = 't,x,y,z'  # カンマで区切った文字列にする．カンマの後スペースを入れないようにする.

# 時刻歴tと解xを一つのndarrayにする
data = np.concatenate(
    [sol.t.reshape(1, len(sol.t)).T, sol.y.T],  # sol.tは1次元配列なので2次元化する
    axis=1
)

# csvで保存
np.savetxt(
    'data.csv',
    data,
    header = header,  # ヘッダーは無くても良い
    comments = '',
    delimiter = ","  # 区切り文字を指定
)

### csvの読み込み
`np.loadtxt()`で読み込める．ただし読み込むcsvファイルにヘッダーがあると少し面倒．  

In [165]:
readed_data_np = np.loadtxt(
    'data.csv',
    delimiter = ',',
    dtype = 'float64',  # デフォルト
    skiprows = 1  # 読み込まない行を上から指定．ヘッダーがある場合は書く．
)

正しく読み込まれてるか確認

In [166]:
readed_data_np  # 読み込んだデータ

array([[ 0.00000000e+00,  0.00000000e+00,  4.00000000e+00,
         2.80000000e+01],
       [ 1.00000000e-02,  3.78749692e-01,  3.96114296e+00,
         2.72707945e+01],
       [ 2.00000000e-02,  7.18002287e-01,  3.92785283e+00,
         2.65746616e+01],
       ...,
       [ 4.99700000e+01, -4.35035429e+00, -7.73222890e+00,
         1.18648249e+01],
       [ 4.99800000e+01, -4.70317650e+00, -8.38082233e+00,
         1.19124333e+01],
       [ 4.99900000e+01, -5.08649937e+00, -9.07820953e+00,
         1.20205558e+01]])

In [167]:
data  # 元のデータ

array([[ 0.00000000e+00,  0.00000000e+00,  4.00000000e+00,
         2.80000000e+01],
       [ 1.00000000e-02,  3.78749692e-01,  3.96114296e+00,
         2.72707945e+01],
       [ 2.00000000e-02,  7.18002287e-01,  3.92785283e+00,
         2.65746616e+01],
       ...,
       [ 4.99700000e+01, -4.35035429e+00, -7.73222890e+00,
         1.18648249e+01],
       [ 4.99800000e+01, -4.70317650e+00, -8.38082233e+00,
         1.19124333e+01],
       [ 4.99900000e+01, -5.08649937e+00, -9.07820953e+00,
         1.20205558e+01]])

### pandasで読み込み
`np.loadtxt`ではヘッダーが読み込まれないので，どの列が何のデータなのかいちいちcsvファイルを開いて確認する必要がある．  
ヘッダーを含めてcsvファイルを読みこむにはpandasが使える．  

In [168]:
import pandas as pd

pandasではデータをデータフレームとして扱う．  
`pd.read_csv('csvファイルのパス')`で読み込む．  

In [169]:
readed_data_pd = pd.read_csv('data.csv')

読み込んだデータフレームを表示してみる．  

In [170]:
readed_data_pd

,t,x,y,z
0,0.00,0.000000,4.000000,28.000000
1,0.01,0.378750,3.961143,27.270795
2,0.02,0.718002,3.927853,26.574662
3,0.03,1.022221,3.904224,25.909113
4,0.04,1.295832,3.893408,25.272026
...,...,...,...,...
4995,49.95,-3.729092,-6.575274,11.921393
4996,49.96,-4.026248,-7.131063,11.870006
4997,49.97,-4.350354,-7.732229,11.864825
4998,49.98,-4.703176,-8.380822,11.912433


どの列が何のデータなのかわかりやすい．  
データフレームはndarrayのように扱える．  

In [171]:
# numpyのブロードキャストの実行例
np.cos(readed_data_pd["x"])

0       1.000000
1       0.929128
2       0.753121
3       0.521472
4       0.271513
          ...   
4995   -0.832330
4996   -0.633556
4997   -0.354178
4998   -0.009212
4999    0.365445
Name: x, Length: 5000, dtype: float64

#### 注意
pandasで読み込むのはかなり遅いです．  
巨大なcsvファイルを読み込むときは使わないほうが良い．  

****
## csv以外の形式で保存
csvやyaml形式で保存するのは，後でpython以外のソフト（Excel, Matlabなど）や異なるOS上で利用したいからです．  
保存したデータをpythonでしか使わないのならば，もっと高速な方法があります．  
### pickeを使う
pickleを使うと`sol`をそのまま保存できる．  

In [172]:
import pickle

`pickle.dump(pythonオブジェクト, ファイル)`で保存  

In [173]:
with open('sol.binaryfile', 'wb') as f:
    pickle.dump(sol, f)

`pickle.load('ファイル')`で保存したpythonオブジェクトを復元する．  

In [174]:
with open('sol.binaryfile', 'rb') as f:
    sol2 = pickle.load(f)

もとのオブジェクトが復元できたことを確認

In [175]:
sol  # 元のオブジェクト

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 5126
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([0.000e+00, 1.000e-02, 2.000e-02, ..., 4.997e+01, 4.998e+01,
       4.999e+01])
 t_events: None
        y: array([[ 0.        ,  0.37874969,  0.71800229, ..., -4.35035429,
        -4.7031765 , -5.08649937],
       [ 4.        ,  3.96114296,  3.92785283, ..., -7.7322289 ,
        -8.38082233, -9.07820953],
       [28.        , 27.27079453, 26.57466159, ..., 11.86482494,
        11.91243329, 12.02055585]])
 y_events: None

In [176]:
sol2  # 復元したオブジェクト

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 5126
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([0.000e+00, 1.000e-02, 2.000e-02, ..., 4.997e+01, 4.998e+01,
       4.999e+01])
 t_events: None
        y: array([[ 0.        ,  0.37874969,  0.71800229, ..., -4.35035429,
        -4.7031765 , -5.08649937],
       [ 4.        ,  3.96114296,  3.92785283, ..., -7.7322289 ,
        -8.38082233, -9.07820953],
       [28.        , 27.27079453, 26.57466159, ..., 11.86482494,
        11.91243329, 12.02055585]])
 y_events: None

他にもnumpy独自のバイナリファイル`.npy`，`.npz`を使う方法があります．  